# Analyse des accidents sur voie publique en FRANCE

Les données correspondes aux accidents corporels sur la circulation publique de France en 2023. Les informations ont été recueillie par les unités de forces publiques qui ont intervenues. 

Les données sont disponibles sous 4 fichiers : 
- caractérstiques : informations sur les caractéristiques de l'accidents  
- lieux : informations sur le lieux de l'accidents
- véhicules : informations sur les véhicules impliqués
- usagers : informations sur les personnes impliqués

Chaque accidents est identifier par un numéro présent dans chaque fichier.

### Objecitf : 

- Classifier les accidents sous différentes catégories pour pouvoir cibler les futures préventions routière, travaux etc...

## Analyse exploratrice des données 

#### Importation

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [5]:
caract = pd.read_csv("acc_routes/caract-2023.csv", sep=";")
lieux = pd.read_csv("acc_routes/lieux-2023.csv",sep=";")
usagers = pd.read_csv("acc_routes/usagers-2023.csv",sep=";")
vehicules = pd.read_csv("acc_routes/vehicules-2023.csv",sep=";")

C:\Users\benfo\AppData\Local\Temp\ipykernel_22288\2705059668.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv("acc_routes/lieux-2023.csv",sep=";")


On vérifie la taille des ensembles

In [9]:
print(caract.shape)
print(lieux.shape)
print(usagers.shape)
print(vehicules.shape)

(54822, 15)
(70860, 18)
(125789, 16)
(93585, 11)


-> On remarque que les ensembles ont des tailles différentes. On explique cela par la dublication d'enregistrements si l'accident concerne plusieurs lieux,véhicules et personnes.

### Explorations générales 